## Data Pre-processing

In [1]:

import pandas as pd
import re
import json

# CSV 파일 불러오기
file_path = 'data/kc/chinese_factories.csv'
df = pd.read_csv(file_path)

df.info()

/var/folders/q0/9z_mzpps4v5f03vr8vrq6xk00000gn/T/ipykernel_89729/1447592657.py:7: DtypeWarning: Columns (0,5,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477291 entries, 0 to 477290
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   certUid                 477288 non-null  object 
 1   certOrganName           477291 non-null  object 
 2   certNum                 477291 non-null  object 
 3   certState               477289 non-null  object 
 4   certDiv                 477291 non-null  object 
 5   certDate                477291 non-null  object 
 6   certChgDate             3 non-null       float64
 7   certChgReason           92728 non-null   object 
 8   firstCertNum            462966 non-null  object 
 9   productName             477237 non-null  object 
 10  brandName               7827 non-null    object 
 11  modelName               477282 non-null  object 
 12  categoryName            415712 non-null  object 
 13  importDiv               421186 non-null  object 
 14  makerName           

### 1. 공장 제조사(makerName) 데이터 전처리

In [2]:
df = df.dropna(subset=['makerName'])
df['makerName_norm'] = df['makerName'].str.upper()


# 고유 업체명 리스트 추출
unique_names = df['makerName_norm'].unique().tolist()

def unify_single_quotes(text):
    # 표준 단일 따옴표와 유사한 문자들 목록 (필요에 따라 추가 가능)
    similar_quotes = "‘’‚‛`´ʼ❛❜＇'"
    # 유사 문자들을 모두 '로 치환하는 정규식 패턴 생성
    pattern = f"[{re.escape(similar_quotes)}]"
    return re.sub(pattern, "'", text)

def remove_korean_in_parentheses(text):
    # ([A-Za-z]+)\([가-힣]*\) : 영문 부분과 괄호 안의 한글 부분을 각각 그룹으로 캡처
    # 대체 문자열은 영문 그룹(\1)만 반환합니다.
    text = re.sub(r'([A-Za-z]+)\([가-힣]+\)', r'\1', text)
    text = re.sub(r'[가-힣]+\(([A-Za-z]+)\)', r'\1', text)
    return re.sub(r'\([가-힣]+\)([A-Za-z]+)',r'\1', text)

def extract_english(text):
   # 정규식 설명:
    # ^             : 문자열 시작
    # ([A-Za-z\s]+) : 영어 알파벳과 공백이 1회 이상 반복되는 그룹 (영어 부분)
    # \s*/\s*       : 슬래시(/) 앞뒤로 공백 허용
    # [가-힣\s]+    : 한글과 공백이 1회 이상 반복되는 부분 (한글 부분)
    pattern = r'^([A-Za-z\s]+)\s*/[가-힣]+[가-힣\s]+'
    match = re.match(pattern, text)
    if match:
        return match.group(1).strip()
    return text

def normalize_company_name(name):
    # 제거할 단어 리스트 (필요에 따라 추가/수정 가능)
    common_words = ['CO LTD', 'LTD', 'INC', 'CORP', 'COMPANY', '유한공사', '주식회사', 'PLC']
    
    # (주) 제거 및 괄호는 공백으로 치환
    name = re.sub(r'\(주\)', '', name)
    name = re.sub(r'\(유\)', '', name)
    name = unify_single_quotes(name)
    name = name.replace("（","(").replace("）",")").replace("[","(").replace("]",")").replace("&"," & ").replace("˚","O").replace('"','').replace("㈜","").replace("_"," ").replace("，"," ").replace("!"," ").replace("‐","-").replace("-"," ")
    
    # 영어(한글) 삭제
    name = remove_korean_in_parentheses(name)
    
    name = name.replace('(', ' ').replace(')', ' ')

    # "CO LTD"가 공백 없이 붙어있거나 공백이 있어도 제거 (문자열 끝에서 제거)
    name = re.sub(r'CO\s*LTD$', '', name, flags=re.IGNORECASE).strip()
    
    # 마지막 단어가 'LIMITED COMPANY' 또는 'COMPANY LIMITED'이면 마지막 두 단어를 'CO LTD'로 대체
    if name.endswith("LIMITED COMPANY") or name.endswith("COMPANY LIMITED"):
        name = ' '.join(name.split()[:-2]) + " CO LTD"
    
    # 불필요한 단어들을 정규식으로 제거
    for word in common_words:
        name = re.sub(r'\b' + re.escape(word) + r'\b', '', name)
    
    # 영어 / 한글 -> 영어 추출
    name = extract_english(name)
    
    # 마지막 단어가 'CO', 'COLTD', 또는 'LIMITED'면 제거 (반복 확인)
    words = name.split()
    while words and words[-1] in {'CO', 'COLTD', 'LIMITED'}:
        words.pop()
    
    return ' '.join(words)

df['makerName_norm'] = df['makerName'].apply(normalize_company_name)

# 정규화된 업체명 딕셔너리 생성: 원본 업체명 -> 정규화 업체명
normalized_names = {name: normalize_company_name(name) for name in unique_names}

with open('norm.json', 'w', encoding='utf-8') as f:
    json.dump(normalized_names, f, ensure_ascii=False, indent=4)


#### [DEBUG]  unique char 조사

In [ ]:
normalized_name_list = [normalize_company_name(name) for name in unique_names]

def extract_unique_characters(names):
    """정제된 이름들의 고유 문자를 추출합니다."""
    unique_chars = set()
    for name in names:
        cleaned_name = normalize_company_name(name)
        unique_chars.update(cleaned_name)  # 고유 문자 추가
    return unique_chars

unique_chars = extract_unique_characters(normalized_name_list)
print(f"Unique characters in normalized names ({len(unique_chars)}):", unique_chars)

#### 유사 제조사 이름 Clustering 

In [3]:
from rapidfuzz import fuzz
import os

# fuzzy matching을 이용한 클러스터링
clusters = {}  # {정규화 업체명 대표: [원본 업체명 목록]}
threshold = 98  # 유사도 임계값 (0~100 사이, 값이 클수록 엄격함)
cluster_file = f'cluster_{threshold}.json'


# 클러스터 파일이 존재하는지 확인하고 불러오기
if os.path.exists(cluster_file):
    with open(cluster_file, 'r', encoding='utf-8') as f:
        clusters = json.load(f)
    print(f"클러스터 데이터가 '{cluster_file}'에서 성공적으로 불러와졌습니다.")
else:
    clusters = {}
    print(f"'{cluster_file}' 파일이 존재하지 않습니다. 빈 클러스터를 생성합니다.")

    for original, norm in normalized_names.items():
        added = False
        # 기존 클러스터와 비교하여 유사도가 임계값 이상이면 같은 그룹에 추가
        for rep in clusters:
            score = fuzz.ratio(norm, rep)
            if score >= threshold:
                clusters[rep].append(original)
                added = True
                break
        # 어떤 클러스터에도 속하지 않으면 새로운 클러스터 생성
        if not added:
            clusters[norm] = [original]

    with open(cluster_file, 'w', encoding='utf-8') as f:
        json.dump(clusters, f, ensure_ascii=False, indent=4)


unique_norm_names = set([ val for key, val in normalized_names.items()])
print(f"before cluster #: {len(unique_norm_names)}")
print(f"total cluster #: {len(clusters)}")

클러스터 데이터가 'cluster_98.json'에서 성공적으로 불러와졌습니다.
before cluster #: 33670
total cluster #: 32869


#### Clustering 결과 mapping

In [4]:
def map_to_cluster_name(name): 
    for key, val in clusters.items():
      if name in val:
          return key
    return name  # 클러스터링에 포함되지 않은 경우 원래 이름 반환


df['makerName_cluster'] = df['makerName'].apply(map_to_cluster_name)
df.head(2)

,certUid,certOrganName,certNum,certState,certDiv,certDate,certChgDate,certChgReason,firstCertNum,productName,...,makerCntryName,importerName,remark,signDate,derivationModels,certificationImageUrls,factories,similarCertifications,makerName_norm,makerName_cluster
0,2558281.0,한국기계전기전자시험연구원(KTC),HU11002-0001B,취소,전기용품안전관리법 대상>안전인증 대상,20010103,NaN,법 위반,HU11002-0001A,전기 스탠드,...,중국,NaN,NaN,20200304.0,NaN,NaN,NaN,NaN,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,PANYU HAODA HARDWARE ELECTRICAL APPLIANC
1,4931443.0,한국산업기술시험원(KTL),SU10010-0002,반납,전기용품 및 생활용품 안전관리법 대상>안전확인대상 전기용품,20010104,NaN,NaN,SU10010-0002,모니터,...,중국,NaN,NaN,20210414.0,NaN,NaN,NaN,NaN,PROVIEW TECHNOLOGY SHENZHEN,PROVIEW TECHNOLOGY SHENZHEN
2,2558303.0,한국기계전기전자시험연구원(KTC),HU07042-1001A,반납,전기용품안전관리법 대상>안전인증 대상,20010108,NaN,자진반납,HU07042-1001A,선풍기,...,중국,NaN,NaN,20200304.0,NaN,NaN,NaN,NaN,WINS INDUSTRIAL,WINS INDUSTRIAL
3,2558309.0,한국기계전기전자시험연구원(KTC),HU10029-1001A,취소,전기용품안전관리법 대상>안전인증 대상,20010109,NaN,법 위반,HU10029-1001A,모니터,...,중국,NaN,NaN,20200304.0,NaN,NaN,NaN,NaN,COMPAL ELECTRONIC CHINA,COMPAL ELECTRONIC CHINA
4,4932512.0,한국산업기술시험원(KTL),SU09004-0001A,반납,전기용품 및 생활용품 안전관리법 대상>안전확인대상 전기용품,20010109,NaN,NaN,SU09004-0001,오디오시스템,...,중국,NaN,NaN,20210414.0,NaN,NaN,NaN,NaN,SAMSUNG ELECTRONICS HUIZHOU,SAMSUNG ELECTRONICS HUIZHOU


### 2. 제품명(productName) 전처리

In [10]:
def remove_unbalanced_parentheses(text):
    # 1. 균형 잡힌 괄호(가장 안쪽부터)를 모두 제거
    pattern_balanced = re.compile(r'\([^()]*\)')
    while re.search(pattern_balanced, text):
        text = re.sub(pattern_balanced, '', text)
    # 2. 아직 남아 있는 미완성 괄호와 그 이후의 모든 내용 제거
    text = re.sub(r'\(.*$', '', text)

    return text.strip()

def rule_based_replace(text):
    # 교체 규칙을 딕셔너리 형태로 정의 (패턴: 치환할 한글)
    rules = {
        r'\bADAPT[EO]R\b': '어댑터',  # ADAPTER, ADAPTOR 둘 다 치환
        r'\bPLAYER\b': '플레이어',
        r'\bRECEIVER\b': '리시버',
        r'\b테블릿': '태블릿',
        r'유사한': '유사',
        r'(?<=[가-힣])\s*와\s*(?=[가-힣])': '',
        r'커패시터': '캐패시터',
        r'레이져': '레이저',
        r'제픔': '제품',
        r'그라인다': '그라인더',
        r'로타리': '로터리',
        r'핼라이드': '할라이드'
    }
    for pattern, replacement in rules.items():
        # 대소문자 구분 없이 치환 (flags=re.IGNORECASE)
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

def normalize_product_name(name):

    # 1. 대문자 변환    
    name = str(name).upper()
    name = name.replace("（","(").replace("）",")").replace("[","(").replace("]",")").replace("_"," ").replace("，"," ").replace("!"," ").replace("‐","-").replace("-"," ").replace(".","·").replace("·"," ").replace(","," ")
    
    name = remove_unbalanced_parentheses(name)  # 괄호와 그 안의 내용 제거
    name = name.replace("(","").replace(")","").replace("및","&")
    name = re.sub(r'\s+', ' ', name)  # 중복 공백 제거
    name = re.sub(r':.*$', '', name).strip()
    name = name.strip()
    
    name = rule_based_replace(name)
    name = re.sub(r'(?<=\w)\s+(?=[가-힣&])|(?<=[가-힣&])\s+(?=\w)', '', name)
    # # 3. 특정 단어 정제
    # common_words = ['기기', '제품', '기타', '전기']
    # for word in common_words:
    #     name = name.replace(word, '')
    
    # 4. 양쪽 공백 제거 후 반환
    return name


df['productName_norm'] = df['productName'].apply(normalize_product_name)

df.head(2)

,certUid,certOrganName,certNum,certState,certDiv,certDate,certChgDate,certChgReason,firstCertNum,productName,...,signDate,derivationModels,certificationImageUrls,factories,similarCertifications,makerName_norm,makerName_cluster,productName_norm,categoryName_norm,certDiv_norm
0,2558281.0,한국기계전기전자시험연구원(KTC),HU11002-0001B,취소,전기용품안전관리법 대상>안전인증 대상,20010103,NaN,법 위반,HU11002-0001A,전기 스탠드,...,20200304.0,NaN,NaN,NaN,NaN,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,전기스탠드,"[백열등기구전기스탠드, 일반조명기구, 조명기기]","[안전인증 대상, 전기용품안전관리법 대상]"
1,4931443.0,한국산업기술시험원(KTL),SU10010-0002,반납,전기용품 및 생활용품 안전관리법 대상>안전확인대상 전기용품,20010104,NaN,NaN,SU10010-0002,모니터,...,20210414.0,NaN,NaN,NaN,NaN,PROVIEW TECHNOLOGY SHENZHEN,PROVIEW TECHNOLOGY SHENZHEN,모니터,"[정보통신사무기기, 모니터]","[전기용품 및 생활용품 안전관리법 대상, 안전확인대상 전기용품]"


In [6]:

## DEBUG
def extract_unique_characters(names):
    """정제된 이름들의 고유 문자를 추출합니다."""
    unique_chars = set()
    for name in names:
        cleaned_name = normalize_company_name(name)
        unique_chars.update(cleaned_name)  # 고유 문자 추가
    return unique_chars

unique_chars = extract_unique_characters(product_norm_list)
print(f"Unique characters in normalized names ({len(unique_chars)}):", unique_chars)

NameError: name 'product_norm_list' is not defined

### 3. 카테고리명(categoryName) 전처리

In [9]:

def rule_based_replace(text):
    # 교체 규칙을 딕셔너리 형태로 정의 (패턴: 치환할 한글)
    rules = {
        r'을 사용하는': '',  
        r'을 이용한': '',  
        r'플래이어': '플레이어',
        r'을 내장한': ' 내장형'

    }
    for pattern, replacement in rules.items():
        # 대소문자 구분 없이 치환 (flags=re.IGNORECASE)
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text


def normalize_category_name(name):
    name = str(name).upper()
    name = name.replace("·","/").replace(",","").replace("/","")
    name = remove_unbalanced_parentheses(name)  # 괄호와 그 안의 내용 제거
    name = rule_based_replace(name)
    name = name.replace(" ","")
    name = list(set([category.strip() for category in (name).split('>')]))

    return name




category_names = list(df['categoryName'].unique())

df['categoryName_norm'] = df['categoryName'].apply(normalize_category_name)
# category_norm = {name: normalize_category_name(name) for name in category_names}

df.head(2)


,certUid,certOrganName,certNum,certState,certDiv,certDate,certChgDate,certChgReason,firstCertNum,productName,...,signDate,derivationModels,certificationImageUrls,factories,similarCertifications,makerName_norm,makerName_cluster,productName_norm,categoryName_norm,certDiv_norm
0,2558281.0,한국기계전기전자시험연구원(KTC),HU11002-0001B,취소,전기용품안전관리법 대상>안전인증 대상,20010103,NaN,법 위반,HU11002-0001A,전기 스탠드,...,20200304.0,NaN,NaN,NaN,NaN,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,전기스탠드,"[백열등기구전기스탠드, 일반조명기구, 조명기기]","[안전인증 대상, 전기용품안전관리법 대상]"
1,4931443.0,한국산업기술시험원(KTL),SU10010-0002,반납,전기용품 및 생활용품 안전관리법 대상>안전확인대상 전기용품,20010104,NaN,NaN,SU10010-0002,모니터,...,20210414.0,NaN,NaN,NaN,NaN,PROVIEW TECHNOLOGY SHENZHEN,PROVIEW TECHNOLOGY SHENZHEN,모니터,"[정보통신사무기기, 모니터]","[전기용품 및 생활용품 안전관리법 대상, 안전확인대상 전기용품]"


### 4. 인증분류(certDiv) 전처리

In [8]:
def normalize_cert_name(name):
    name = list(set([category.strip() for category in (name).split('>')]))

    return name

df['certDiv_norm'] = df['certDiv'].apply(normalize_cert_name)

df.head(2)


,certUid,certOrganName,certNum,certState,certDiv,certDate,certChgDate,certChgReason,firstCertNum,productName,...,signDate,derivationModels,certificationImageUrls,factories,similarCertifications,makerName_norm,makerName_cluster,productName_norm,categoryName_norm,certDiv_norm
0,2558281.0,한국기계전기전자시험연구원(KTC),HU11002-0001B,취소,전기용품안전관리법 대상>안전인증 대상,20010103,NaN,법 위반,HU11002-0001A,전기 스탠드,...,20200304.0,NaN,NaN,NaN,NaN,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,전기스탠드,"[백열등기구전기스탠드, 일반조명기구, 조명기기]","[안전인증 대상, 전기용품안전관리법 대상]"
1,4931443.0,한국산업기술시험원(KTL),SU10010-0002,반납,전기용품 및 생활용품 안전관리법 대상>안전확인대상 전기용품,20010104,NaN,NaN,SU10010-0002,모니터,...,20210414.0,NaN,NaN,NaN,NaN,PROVIEW TECHNOLOGY SHENZHEN,PROVIEW TECHNOLOGY SHENZHEN,모니터,"[정보통신사무기기, 모니터]","[전기용품 및 생활용품 안전관리법 대상, 안전확인대상 전기용품]"


### 5. 공장지역이름 추가

In [12]:

# 대표적인 중국 제조지역 리스트 (모두 대문자)
regions = {
    
    "SHENZHEN": "심천",
    "SHANGHAI": "상해",
    "BEIJING": "베이징",
    "GUANGZHOU": "광저우",
    "CHENGDU": "청두",
    "TIANJIN": "텐진",
    "DONGGUAN": "동관",
    "WUHAN": "우한",
    "CHONGQING": "충칭",
    "SUZHOU": "쑤저우",
    "NINGBO": "닝보",
    "XIAMEN": "샤먼",
    "ZHONGSHAN": "중산",
    "BEIHAI": "베이하이",
    "ZHUHAI": "주하이",
    "FOSHAN": "포산",
    "NANJING": "난징",
    "CHANGZHOU": "창저우",
    "HANGZHOU": "항저우",
    "HUIZHOU": "후이저우",
    "QINGDAO": "칭다오",
    
}

def map_region(maker_name):
    for key in regions.keys():
        if key in maker_name:
            return regions[key]
    return None  # 키가 없으면 None 반환


df['region'] = df['makerName_cluster'].apply(map_region)

df.head(2)


,certUid,certOrganName,certNum,certState,certDiv,certDate,certChgDate,certChgReason,firstCertNum,productName,...,derivationModels,certificationImageUrls,factories,similarCertifications,makerName_norm,makerName_cluster,productName_norm,categoryName_norm,certDiv_norm,region
0,2558281.0,한국기계전기전자시험연구원(KTC),HU11002-0001B,취소,전기용품안전관리법 대상>안전인증 대상,20010103,NaN,법 위반,HU11002-0001A,전기 스탠드,...,NaN,NaN,NaN,NaN,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,PANYU HAODA HARDWARE ELECTRICAL APPLIANC,전기스탠드,"[백열등기구전기스탠드, 일반조명기구, 조명기기]","[안전인증 대상, 전기용품안전관리법 대상]",None
1,4931443.0,한국산업기술시험원(KTL),SU10010-0002,반납,전기용품 및 생활용품 안전관리법 대상>안전확인대상 전기용품,20010104,NaN,NaN,SU10010-0002,모니터,...,NaN,NaN,NaN,NaN,PROVIEW TECHNOLOGY SHENZHEN,PROVIEW TECHNOLOGY SHENZHEN,모니터,"[정보통신사무기기, 모니터]","[전기용품 및 생활용품 안전관리법 대상, 안전확인대상 전기용품]",심천


## 데이터 Export 

In [ ]:
select_columns = ["makerName_cluster", "makerName", "region", "certOrganName", "certNum", "certState", "certDate", "productName_norm", "modelName", "brandName", "importDiv", "makerCntryName", "categoryName_norm", "certDiv_norm"]

df_final = df[select_columns]

OUTPUT_FILE = "prep_china_factory.csv"
df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

print(f"선택한 열이 '{OUTPUT_FILE}'로 저장되었습니다.")

선택한 열이 'prep_china_fact_data.csv'로 저장되었습니다.


In [ ]:
result_df = df.groupby('makerName_cluster').agg({
    'makerName': lambda x: list(x.unique()),  # 중복 제거한 리스트
    'certNum': lambda x: list(x),              # certNum도 리스트로 집계 (필요 시)
                                  
}).reset_index()

json_file_path = 'manufacturer_v1.json'  # 저장할 JSON 파일 경로

result_df.to_json(json_file_path, orient='records', force_ascii=False, indent=4)

### DEBUG

In [ ]:
# Case 1. 띄어쓰기가 없을 때
filtered_data = {key: value for key, value in clusters.items() if re.fullmatch(r'[A-Za-z]+', key)}
with open(f'cluster_nospace_{threshold}.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)
print(f"case 1. 띄어쓰기 없는 경우: {len(filtered_data.keys())}")

# Case 2. 한글회사 + 같은 영어 회사 이름
def contains_both_languages(s):
    return bool(re.search(r'[A-Za-z]', s)) and bool(re.search(r'[가-힣]', s))
filtered_data = { key: values for key, values in clusters.items()  if any(contains_both_languages(item) for item in values) }
with open(f'cluster_diffkor_sameeng_{threshold}.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)
print(f"case 2. 한글회사 + 같은 영어 회사 이름 경우: {len(filtered_data.keys())}")

# Case 3. 한글 + 영어 섞인 회사 이름
filtered_data = { key: value for key, value in clusters.items() if contains_both_languages(key) }
with open(f'cluster_mixkoreng_{threshold}.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)
print(f"case 3. 한글+영어 회사 이름 경우: {len(filtered_data.keys())}")

# Case 4. 길이가 2 이상인 회사 이름
filtered_data = { key: value for key, value in clusters.items() if len(value) > 1 }
with open(f'cluster_len2_{threshold}.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)
print(f"case 4. 후보군이 2 이상인 회사 이름 경우: {len(filtered_data.keys())}")

# Case 5. 길이가 1인 회사 이름
filtered_data = { key: value for key, value in clusters.items() if len(value) == 1 }
with open(f'cluster_single_{threshold}.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)
print(f"case 4. 후보군이 하나인 회사 이름 경우: {len(filtered_data.keys())}")



with open(f'cluster_{threshold}.json', 'w', encoding='utf-8') as f:
    json.dump(clusters, f, ensure_ascii=False, indent=4)